# Association Rules
## Submitted By: Anubhav Gupta
## Date: 02/06/2018


## Data Transformations in SQL

##### Identified the following features of the pos.tansact table in the database:
* C1 == SKU
* C2 == STORE
* C3 == REGISTER
* C4 == TRANNUM
* C6 == SALEDATE
* C7 == STYPE

##### Ran the following 1-time queries to Extract, Transform and Load into the schema. Notice that a unique STORE, REGISTER, TRANNUM and SALEDATE combination gives us 1 basket. Also, we pick only the 'Purchase' transactions.

```
CREATE TABLE agq3445_schema.trnsact_baskets AS
SELECT c1 AS sku, 
    NULLIF(c2, '')::int as store,
    to_timestamp(c6, 'YYYY-MM-DD') as datetran,
    c2||','||c3||','||c4||','||c6 as basketId 
        FROM pos.trnsact where c7='P';
```

##### Added the following indexes to make the queries run fast. We cluster the trnsact_baskets table by datetran field so that we can do fast range queries

```
CREATE INDEX idx_datetran
    ON agq3445_schema.trnsact_baskets USING btree
    (datetran ASC NULLS LAST)

ALTER TABLE agq3445_schema.trnsact_baskets
    CLUSTER ON idx_datetran;

CREATE INDEX idx_sku
    ON agq3445_schema.trnsact_baskets USING btree
    (sku ASC NULLS LAST);
    
CREATE INDEX idx_store
    ON agq3445_schema.trnsact_baskets USING btree
    (store ASC NULLS LAST);
```

##### Once the heavy lifting is done, we proceed to perform exploratory analysis on the new table

In [ ]:
#Importing necessary libraries and connecting to the database
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2 as pg
from orangecontrib.associate.fpgrowth import *
import properties

#Add properties file of the following format:
#host="<host_name>"
#dbname="<db_name>"
#user="<username>"
#password="<password>"

conn_string = "host='%s' dbname='%s' user='%s' password='%s'" % (properties.host, properties.dbname, properties.user, properties.password)
print("Connecting to database\n%s" % (conn_string))
conn = pg.connect(conn_string)

def execute_query(query_str):
    return pd.read_sql(query_str, con = conn)

## Exploratory Analysis

In [ ]:
#Finding the range of dates for which transactions are available
df = execute_query("select min(datetran), max(datetran) from agq3445_schema.trnsact_baskets")
df.head()

In [ ]:
#For each date find and plot the number of transactions across all the stores
df = execute_query("select datetran, count(*) from agq3445_schema.trnsact_baskets group by datetran")

plt.figure()
dates = [pd.to_datetime(d) for d in df['datetran']]
plt.scatter(dates, df['count'])
plt.xlabel('Date')
plt.ylabel('Number of Transactions')
plt.show()

#There doesn't seem to be a decipherable pattern in the counts, so we can do sampling according to the dates

In [ ]:
#Finding total number of stores

df = execute_query("select count(distinct(store)) as stores from pos.strinfo")
print(df)

In [ ]:
#Finding number of stores with transaction data

df = execute_query("select count(distinct(store)) from agq3445_schema.trnsact_baskets")
print(df)

#This means we have transaction data for 332/453 stores available

In [ ]:
#Finding number of stores with respective transaction counts

df = execute_query("select store, count(*) as num_transactions from agq3445_schema.trnsact_baskets group by store")
print(df.head())


In [ ]:
#Finding number of skus defined for the dataset

df = execute_query("select count(distinct(sku)) as num_skus from pos.skuinfo")
print(df)

## Analysis

## Performing data subsetting by selecting transactions across all the stores for August 2015.

In [ ]:
df = execute_query("SELECT basketid, sku FROM agq3445_schema.trnsact_baskets where datetran > '2005-08-01 00:00:00-05'")
df.head()

In [ ]:
#Doing some exploration on the subset

num_baskets = len(df.basketid.unique())
unique_skus = len(df.sku.unique())
num_transactions = len(df.index)

print("Total number of transactions: ", num_transactions)
print("Total number of baskets: ", num_baskets)
print("Average basket size in the data subset: " + str(int(num_transactions/num_baskets)))
print("Total number of unique skus in the transaction data subset: ", unique_skus)

In [ ]:
#Reshaping each basket into list of SKUs
baskets_set = list(df['sku'].groupby(df['basketid']))
for i in range(len(baskets_set)):
    indexed_transaction = list(baskets_set[i][1:])[0]
    baskets_set[i] = [trans for trans in indexed_transaction]

In [ ]:
from orangecontrib.associate.fpgrowth import *

itemsets = dict(frequent_itemsets(baskets_set, 600))
rules = list(association_rules(itemsets, 0.2))
rules_stats = rules_stats(rules, itemsets, num_baskets)
final_rules_stats = list(rules_stats)

## Post Processing

In [ ]:
#Reading sku information from the table
skus = pd.read_sql("SELECT * from pos.skuinfo", con=conn)

In [ ]:
import re
def get_sku_details(set_sku):
    str_sku = []
    str_dept = []
    str_brand = []
    for sku in set_sku:
        str_sku.append(re.sub("\D", "",sku))
        if not skus.loc[skus['sku'] == int(sku)].empty: 
            str_dept.append(str((skus.loc[skus['sku'] == int(sku)])['dept'].values[0]))
            str_brand.append(str((skus.loc[skus['sku'] == int(sku)])['brand'].values[0]).strip())
    return str_sku, str_dept, str_brand 

final_rules_stats_new = []
for i in range(len(final_rules_stats)): 
    str_skus_lhs, str_dept_lhs, str_brand_lhs = get_sku_details(final_rules_stats[i][0])
    str_skus_rhs, str_dept_rhs, str_brand_rhs = get_sku_details(final_rules_stats[i][1])
    final_rules_stats_new.append([str_skus_lhs, str_dept_lhs, str_brand_lhs, str_skus_rhs, str_dept_rhs, str_brand_rhs,
                            final_rules_stats[i][2], final_rules_stats[i][3], final_rules_stats[i][6]])


final_output = pd.DataFrame(final_rules_stats_new, columns = ["Antecedent_SKUs","Antecedent_Dept","Antecedent_Brand", "Consequent_SKUs","Consequent_Dept","Consequent_Brand", "Support", "Confidence","Lift"])
final_output = final_output.sort_values(by='Lift', ascending=False)     
final_output.to_csv("output.csv", header="true", index=False)
final_output
        